### Parameters

In [1]:
SEED = 123  # modifiable seed
CLF_SS = 2      # sub-sample model types for faster run
TARGETS = -1    # which target (0-4) to predict; -1 for all

### Imports

In [2]:
import numpy as np  
import pandas as pd 
import pickle

In [3]:
import multiprocessing
from joblib import Parallel, delayed

In [4]:
from collections import Counter
import datetime as datetime

In [5]:
import gc
import psutil
import sys

In [6]:
import matplotlib.pyplot as plt
import seaborn as sns
from pylab import rcParams
rcParams['figure.figsize'] = (15,5.5)

pd.options.display.max_rows = 150

In [7]:
start = datetime.datetime.now()

if SEED < 0:
    np.random.seed(datetime.datetime.now().microsecond)
else:
    np.random.seed(SEED)

### Data Loading

In [8]:
path = '/kaggle/input/trends-assessment-prediction/'

loading =  pd.read_csv(path+ '/' + 'loading.csv').set_index('Id')
fnc =  pd.read_csv(path+ '/' + 'fnc.csv').set_index('Id')
assert len(loading) == len(fnc)


In [9]:
y_data =  pd.read_csv(path+ '/' + 'train_scores.csv').set_index('Id')

data = pd.concat((loading, fnc,  ), axis = 'columns')  
test_data = data[~data.index.isin(y_data.index)]

X = data.loc[y_data.index] 
y = y_data 
groups = np.random.randint(0, 5, len(y))

### Model Setup

In [10]:
from sklearn.model_selection import RandomizedSearchCV, RepeatedKFold, KFold, ShuffleSplit
from sklearn.svm import SVR, NuSVR
from sklearn.linear_model import ElasticNet, Ridge, Lasso
from sklearn.model_selection import ParameterSampler
from sklearn.metrics import make_scorer, mean_absolute_error
from sklearn.base import clone
from sklearn.preprocessing import StandardScaler, FunctionTransformer 

In [11]:
nusvr_params = {
    'kernel': [  'rbf',  ] , 
    'C': [ 1, 2, 3, 5, 7, 10, 15, 20, 30, 50, 60,80, 100, 130, 200  ],
    'gamma': [ 'scale'], 
    'nu': [   0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 1] }

def trainNuSVR(x, y, groups, cv = 0, n_jobs = -1, **kwargs):
    clf = NuSVR(cache_size=1000)
    params = nusvr_params        
    return trainModel(x, y, groups, clf, params, cv, n_jobs,  **kwargs)

In [12]:
enet_params = { 'alpha': [  1e-5, 2e-5, 5e-5, 1e-4, 2e-4, 5e-4, 1e-3, 2e-3, 5e-3, 1e-2, 3e-2, 0.1, 0.3,   ],
                'l1_ratio': [ 0, 0.01, 0.02, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.7, 0.95, 0.97, 0.98, 0.99, 1,   ]}

def trainENet(x, y, groups, cv = 0, n_jobs = -1, **kwargs):
    clf = ElasticNet(normalize = True, selection = 'random', max_iter = 1000 )
    return trainModel(x, y, groups, clf, enet_params, cv, n_jobs, **kwargs)

In [13]:
def fnae(y_true, y_pred):
    valid = ~np.isnan(y_true)
    y_true = y_true[valid]
    y_pred = y_pred[valid]
    return np.sum(np.abs(y_true - y_pred))/np.sum(y_true)

fnae_scorer = make_scorer(fnae, greater_is_better = False)

In [14]:
def trainModel(x, y, groups, clf, params, cv = 0, n_jobs = None, 
                   verbose=0, splits=None, **kwargs):
    if n_jobs is None:
        n_jobs = -1    

    n_iter = 40
        
    folds = ShuffleSplit(n_splits = 7, train_size = 0.75, test_size = 0.20)
    clf = RandomizedSearchCV(clf, params, cv = folds, n_iter = n_iter, 
                            verbose = 1, n_jobs = n_jobs, scoring = fnae_scorer)
    
    f = clf.fit(x, y, groups)
    
    print(pd.DataFrame(clf.cv_results_['mean_test_score'])); print();  
    best = clf.best_estimator_;  print(best)
    print("Best Score: {}".format(np.round(clf.best_score_,4)))
    
    return best

In [15]:
def cleanX(X, target):
    X = X.copy()
    
    for col in fnc.columns:
        X[col] = X[col] / 500
       
    return X;

In [16]:
def runBag(n = 3, model_type = trainENet, data = None, **kwargs):
    start_time = datetime.datetime.now(); 
    
    X, y, groups = data

    valid = ~y.isnull()
    X = X[valid]; y = y[valid]; groups = groups[valid]
    
    if 'target' in kwargs:
        X = cleanX(X, kwargs['target'])
    
    group_list = [*dict.fromkeys(groups)]   
    group_list.sort()
    
    clfs = []; preds = []; ys=[]; datestack = []
    for group in group_list:
        g = gc.collect()
        x_holdout = X[groups == group]
        y_holdout = y[groups == group]
        x_train = X[groups != group]
        y_train = y[groups != group]
        
        groups_train = groups[groups != group]

        model = model_type 
        clf = model(x_train, y_train, groups_train, **kwargs) 
        clfs.append(clf)

        predicted = clf.predict(x_holdout)
        print("{}: {:.4f}".format(group,
              fnae(y_holdout, predicted)  ) )
        
        preds.append(predicted)
        ys.append(y_holdout)
    
    y_pred = np.concatenate(preds)
    y_ho = np.concatenate(ys) 

    end_time = datetime.datetime.now(); 
    print("\nModel Bag Time: {}\n".format(str(end_time - start_time).split('.', 2)[0] ))
    return clfs

In [17]:
def trainBaseClfs(clfs, clf_names, data, target = None, **kwargs):
    start_time = datetime.datetime.now(); 
    
    X, y, groups = data
    
    X = cleanX(X, target)
    
    group_list = [*dict.fromkeys(groups)]   
    group_list.sort()
    
    X_ordered = []; y_ordered = []; groups_ordered =[]  
    all_base_clfs = []; base_preds = [[] for i in range(0, 5 * len(clfs))]; 
    for group in group_list:
        print("Training Fold {} of {}:".format(group, len(group_list)))
        np.random.seed(SEED)
        
        x_holdout = X[groups == group]
        y_holdout = y[groups == group]
        x_train = X[groups != group]
        y_train = y[groups != group]

        y_idx = ALL_TARGETS.index(target)
        
        X_ordered.append(x_holdout)
        y_ordered.append(y_holdout)
        groups_ordered.append(groups[groups == group])
        
        base_clfs = []
        for idx, clf in enumerate(clfs):
            base_clfs.append(clone(clf))
        
        def train_model(model, X, y):
            ss = (~pd.DataFrame(y).isnull().any(axis=1))
            model.fit(X[ss], y[ss]); return model
        
        base_clfs = Parallel(n_jobs=4)(delayed(train_model)(model, x_train, y_train[y_var]) for model in base_clfs)
        all_base_clfs.append(base_clfs)
        
        def predict_model(model, X):
            o = model.predict(X); return o    
        preds = Parallel(n_jobs=4)(delayed(predict_model)(model, x_holdout) for model in base_clfs)
        
        
        pidx = 0; clf_pred_names = []
        for idx, clf in enumerate(base_clfs):   
            print("{:.4f} for {}".format( 
                      fnae(y_holdout[target], preds[idx]), clf_names[idx]  ) )
            base_preds[pidx].append(preds[idx]); pidx+=1;
            clf_pred_names.append(clf_names[idx])
            
        print("\nTime Elapsed: {}\n".format(str(datetime.datetime.now() - start_time).split('.', 2)[0] ))

    base_preds = base_preds[:len(clf_pred_names)]
    for idx in range(0, len(base_preds)):
        base_preds[idx] = np.concatenate(base_preds[idx])

    
    print("\Base Classifier Train Time: {}\n".format(str(datetime.datetime.now() - start_time).split('.', 2)[0] ))
    return (all_base_clfs, base_preds, clf_pred_names, 
        pd.concat(X_ordered), pd.concat(y_ordered), np.concatenate(groups_ordered))

In [18]:
def Lassos():
    clfs = []; clf_names = []
    lassos =  [1e-5, 3e-5, 1e-4,  3e-4,  0.001, 0.003,  0.01,  0.03,  0.1,  0.3,  1, ]
    for l in lassos:
        clfs.append(Lasso(alpha = l,  selection = 'random', max_iter = 1000))
        clf_names.append('Lasso alpha={}'.format(l))
        if CLF_SS > 1:
            clfs.append(clfs[-1]); clf_names.append(clf_names[-1])
 
    return clfs, clf_names

In [19]:
def Ridges():
    clfs = []; clf_names = []
    ridges =  [1e-4,  3e-4,  0.001, 0.003,  0.01,  0.03,  0.1,  0.3,  1,  3,  10,    ]
    for r in ridges:
        clfs.append(Ridge(alpha = r))
        clf_names.append('Ridge alpha={}'.format(r))
        if CLF_SS > 1:
            clfs.append(clfs[-1]); clf_names.append(clf_names[-1])

    return clfs, clf_names

In [20]:
def SVRs():
    clfs = []; clf_names = []
    svrs =  ([0.2, 1, 7,30, 50, 60, 100], [1, 3, 7]) 
    for c in svrs[0]:
        for e in svrs[1]:
            clfs.append(SVR(C = c, epsilon = e, cache_size=1000))
            clf_names.append('SVR C={}, epsilon={}'.format(c,e))
            
    return clfs, clf_names

In [21]:
def ENets():
    clfs = []; clf_names = []
    enets = ([1e-4, 3e-4, 1e-3, 3e-3  ], [ 0, 0.1, 0.5, 0.9, 0.95, 1],[50,100,500,1000]) 
    for a in enets[0]:
        for l in enets[1]:
            for n in enets[2]:
                clfs.append(ElasticNet(alpha = a, l1_ratio = l,
                         normalize = False, selection = 'random', 
                         max_iter = n))
                clf_names.append('Enet alpha={}, l1_ratio={}, n_iter={}'.format(a,l,n))
 
    for a in enets[0]:
        for l in enets[1]:
            for n in enets[2]:
                clfs.append(ElasticNet(alpha = a, l1_ratio = l,
                         normalize = True, selection = 'random', 
                         max_iter = n ))
                clf_names.append('Enet-n alpha={}, l1_ratio={}, n_iter={}'.format(a,l,n))
            
    return clfs, clf_names

In [22]:
def getBaseClfs(y_var):
    idx = ALL_TARGETS.index(y_var)

    clfs = []
    clf_names = []
    
    model_sets =  [SVRs(), ENets(), Lassos(), Ridges()]
    for model_set in model_sets:
        clfs.extend(model_set[0])
        clf_names.extend(model_set[1])
   

    return clfs[::CLF_SS], clf_names[::CLF_SS];


In [23]:
ALL_TARGETS = y.columns.to_list()  
if isinstance(TARGETS, list):
    targets = [ALL_TARGETS[i] for i in TARGETS]
elif TARGETS is not None and TARGETS >= 0:
    targets = ALL_TARGETS[TARGETS: TARGETS + 1]
else:
    targets = ALL_TARGETS
# print(targets)

In [24]:
def metaFilter(X):
    return X[[c for c in X.columns if c not in data.columns or c in loading.columns ]] 

### Train Models

In [25]:
all_clfs = []; all_raw_base_clfs = []; all_base_clfs = []; scalers = []
for idx, y_var in enumerate(targets):
    print('---Training Models for {}---\n'.format(y_var))
       
    
    # train base classifiers
    raw_base_clfs, base_clf_names = getBaseClfs(y_var)
    all_raw_base_clfs.append((raw_base_clfs, base_clf_names))
    
    base_clfs, base_clf_preds, base_clf_names, Xe, ye, ge = \
                    trainBaseClfs(raw_base_clfs, base_clf_names, 
                                  data = (X, y, groups), 
                                  target=y_var, )
    Xe = pd.concat( (Xe, pd.DataFrame( dict(zip(base_clf_names, base_clf_preds)), index=Xe.index) ),
                     axis = 'columns')
    
    all_base_clfs.append((base_clfs, base_clf_preds, base_clf_names, Xe, ye, ge ))
    
    
    # train meta model
    s = FunctionTransformer()
    if y_var == 'age':
        meta_model = trainNuSVR
    else:
        meta_model = trainENet
     
    s.fit(metaFilter(Xe))
    scalers.append(s)
    
    all_clfs.append( runBag(data = (s.transform(metaFilter(Xe)), ye[y_var], ge), # target=y_var,
                                   model_type = meta_model) )
    # run

---Training Models for age---

Training Fold 0 of 5:
0.1834 for SVR C=0.2, epsilon=1
0.1878 for SVR C=0.2, epsilon=7
0.1638 for SVR C=1, epsilon=3
0.1541 for SVR C=7, epsilon=1
0.1553 for SVR C=7, epsilon=7
0.1491 for SVR C=30, epsilon=3
0.1480 for SVR C=50, epsilon=1
0.1498 for SVR C=50, epsilon=7
0.1480 for SVR C=60, epsilon=3
0.1479 for SVR C=100, epsilon=1
0.1492 for SVR C=100, epsilon=7
0.1921 for Enet alpha=0.0001, l1_ratio=0, n_iter=100
0.1921 for Enet alpha=0.0001, l1_ratio=0, n_iter=1000
0.1903 for Enet alpha=0.0001, l1_ratio=0.1, n_iter=100
0.1903 for Enet alpha=0.0001, l1_ratio=0.1, n_iter=1000
0.1806 for Enet alpha=0.0001, l1_ratio=0.5, n_iter=100
0.1806 for Enet alpha=0.0001, l1_ratio=0.5, n_iter=1000
0.1636 for Enet alpha=0.0001, l1_ratio=0.9, n_iter=100
0.1636 for Enet alpha=0.0001, l1_ratio=0.9, n_iter=1000
0.1602 for Enet alpha=0.0001, l1_ratio=0.95, n_iter=100
0.1602 for Enet alpha=0.0001, l1_ratio=0.95, n_iter=1000
0.1537 for Enet alpha=0.0001, l1_ratio=1, n_iter=100

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   45.6s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done 280 out of 280 | elapsed:  4.8min finished


           0
0  -0.141215
1  -0.141161
2  -0.140909
3  -0.140861
4  -0.140749
5  -0.140611
6  -0.141053
7  -0.140850
8  -0.140635
9  -0.141138
10 -0.140762
11 -0.141653
12 -0.140950
13 -0.140978
14 -0.140958
15 -0.141068
16 -0.141012
17 -0.140700
18 -0.140730
19 -0.141104
20 -0.141117
21 -0.140748
22 -0.141460
23 -0.141568
24 -0.140831
25 -0.141568
26 -0.140612
27 -0.141117
28 -0.140741
29 -0.140908
30 -0.141480
31 -0.140681
32 -0.141356
33 -0.140900
34 -0.141160
35 -0.141095
36 -0.140952
37 -0.140790
38 -0.141436
39 -0.141498

NuSVR(C=50, cache_size=1000, coef0=0.0, degree=3, gamma='scale', kernel='rbf',
      max_iter=-1, nu=0.6, shrinking=True, tol=0.001, verbose=False)
Best Score: -0.1406
0: 0.1474
Fitting 7 folds for each of 40 candidates, totalling 280 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   33.4s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done 280 out of 280 | elapsed:  4.5min finished


           0
0  -0.141098
1  -0.142100
2  -0.141407
3  -0.141755
4  -0.141522
5  -0.141559
6  -0.140993
7  -0.141327
8  -0.141823
9  -0.141097
10 -0.141186
11 -0.141757
12 -0.140984
13 -0.141101
14 -0.141983
15 -0.141768
16 -0.141913
17 -0.141171
18 -0.141192
19 -0.141075
20 -0.141449
21 -0.141147
22 -0.141476
23 -0.142248
24 -0.141698
25 -0.142022
26 -0.141528
27 -0.141423
28 -0.141474
29 -0.142170
30 -0.141025
31 -0.141111
32 -0.141323
33 -0.141231
34 -0.141093
35 -0.141084
36 -0.141102
37 -0.141395
38 -0.141157
39 -0.141301

NuSVR(C=30, cache_size=1000, coef0=0.0, degree=3, gamma='scale', kernel='rbf',
      max_iter=-1, nu=0.8, shrinking=True, tol=0.001, verbose=False)
Best Score: -0.141
1: 0.1416
Fitting 7 folds for each of 40 candidates, totalling 280 fits


...

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  76 tasks      | elapsed:    4.1s
[Parallel(n_jobs=-1)]: Done 280 out of 280 | elapsed:   16.6s finished


           0
0  -0.178668
1  -0.175421
2  -0.175478
3  -0.176611
4  -0.176044
5  -0.175641
6  -0.179553
7  -0.175737
8  -0.176190
9  -0.175304
10 -0.175946
11 -0.179553
12 -0.175481
13 -0.179547
14 -0.175202
15 -0.175221
16 -0.179553
17 -0.175674
18 -0.175279
19 -0.176415
20 -0.176427
21 -0.175223
22 -0.179553
23 -0.178912
24 -0.175254
25 -0.176013
26 -0.176778
27 -0.175258
28 -0.175932
29 -0.176699
30 -0.175475
31 -0.176337
32 -0.176170
33 -0.175961
34 -0.176127
35 -0.176113
36 -0.178756
37 -0.175813
38 -0.175753
39 -0.179553

ElasticNet(alpha=1e-05, copy_X=True, fit_intercept=True, l1_ratio=0.5,
           max_iter=1000, normalize=True, positive=False, precompute=False,
           random_state=None, selection='random', tol=0.0001, warm_start=False)
Best Score: -0.1752
3: 0.1790
Fitting 7 folds for each of 40 candidates, totalling 280 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    3.1s
[Parallel(n_jobs=-1)]: Done 268 tasks      | elapsed:   19.7s
[Parallel(n_jobs=-1)]: Done 273 out of 280 | elapsed:   20.5s remaining:    0.5s
[Parallel(n_jobs=-1)]: Done 280 out of 280 | elapsed:   20.9s finished


           0
0  -0.175326
1  -0.176955
2  -0.175871
3  -0.177087
4  -0.176757
5  -0.179546
6  -0.177331
7  -0.176679
8  -0.175481
9  -0.176944
10 -0.175336
11 -0.176155
12 -0.176903
13 -0.175445
14 -0.179546
15 -0.175826
16 -0.176707
17 -0.175714
18 -0.176870
19 -0.175717
20 -0.176348
21 -0.177338
22 -0.179546
23 -0.176331
24 -0.176903
25 -0.176848
26 -0.176456
27 -0.176169
28 -0.175864
29 -0.176950
30 -0.176132
31 -0.176509
32 -0.179546
33 -0.175855
34 -0.175818
35 -0.176856
36 -0.176894
37 -0.176917
38 -0.175583
39 -0.175445

ElasticNet(alpha=2e-05, copy_X=True, fit_intercept=True, l1_ratio=0.95,
           max_iter=1000, normalize=True, positive=False, precompute=False,
           random_state=None, selection='random', tol=0.0001, warm_start=False)
Best Score: -0.1753
4: 0.1701

Model Bag Time: 0:01:21



/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 276.2021362797823, tolerance: 67.27311098587201
  positive)


### Prediction Code

In [26]:
def predictBag(X, y, groups, clfs, target = None):
    start_time = datetime.datetime.now(); 

    valid = ~y.isnull()
    X = X[valid]; y = y[valid]; groups = groups[valid]
    
    if target is not None:
        X = cleanX(X, target)
    
    group_list = [*dict.fromkeys(groups)]   
    group_list.sort()

    preds = []; ys=[]; datestack = []
    for idx, group in enumerate(group_list):
        g = gc.collect()
        x_holdout = X[groups == group]
        y_holdout = y[groups == group]
  
        y_pred = clfs[idx].predict(x_holdout)    
        preds.append(y_pred)
        ys.append(y_holdout)
    
        print("{}: {:.4f}".format(group,
              fnae(y_holdout, y_pred) ) )
        
    y_pred = np.concatenate(preds)
    y_true = np.concatenate(ys) 
    
    print("\Bag Prediction Time: {}\n".format(str(datetime.datetime.now() - start_time).split('.', 2)[0] ))
    return y_pred, y_true

In [27]:
def predictAll(X_test, all_base_clfs, all_clfs):
    start_time = datetime.datetime.now(); 
        
    def predict_model(model, X):
        o = model.predict(X)
        return o    
    
    all_preds = pd.DataFrame(columns = targets, index=X_test.index)
    for tidx, y_var in enumerate(targets): # loop over targets
        print(y_var)
        Xi = cleanX(X_test, y_var)
        base_clfs = all_base_clfs[tidx][0]
         

        preds = []; 
        for g_idx, g_clfs in enumerate(base_clfs): # loop over groups
            print(g_idx)
            preds.append(Parallel(n_jobs=4)(delayed(predict_model)(model, Xi) for model in g_clfs))
        print("\Base Classifier Prediction Time: {}\n".format(str(datetime.datetime.now() - start_time).split('.', 2)[0] ))


        c_preds = []; sub_preds = np.zeros((len(preds), len(Xi)))
        for c_idx in range(0, len(preds[0])):  
            if len(preds[0][c_idx].shape) > 1: 
                for t_idx in range(0, preds[0][c_idx].shape[1]):
                    for g_idx, this_pred_group in enumerate(preds):  
                        sub_preds[g_idx, :] = this_pred_group[c_idx][:, t_idx]
                    c_preds.append(np.mean( sub_preds, axis = 0))  
            else:
                for g_idx, this_pred_group in enumerate(preds): 
                    sub_preds[g_idx, :] = this_pred_group[c_idx]
                c_preds.append(np.mean( sub_preds, axis = 0)) 

        Xf = pd.concat( (Xi, pd.DataFrame( dict(zip(all_base_clfs[tidx][2], c_preds)), index=Xi.index) ),
                     axis = 'columns')
        print("\nTime Elapsed: {}\n".format(str(datetime.datetime.now() - start_time).split('.', 2)[0] ))
 

        s = scalers[tidx]
        print('\nrunning stacker')
        pred = Parallel(n_jobs=4)(delayed(predict_model)(model, s.transform(metaFilter(Xf))) 
                                                       for model in all_clfs[tidx])
        sub_preds = np.zeros((len(all_clfs[tidx]), len(Xi)))
        for g_idx, clf in enumerate(all_clfs[tidx]):
            sub_preds[g_idx, :] = pred[g_idx]
        all_preds[y_var] = np.mean(sub_preds, axis = 0)


    end_time = datetime.datetime.now(); 
    print("\Prediction Time: {}\n".format(str(end_time - start_time).split('.', 2)[0] ))
    return all_preds, Xf

### Show Scores by Fold

In [28]:
y_preds = pd.DataFrame(index = X.index)
y_trues = y_preds.copy()
scores = pd.DataFrame(index = targets, columns = ['score'])
for idx, y_var in enumerate(targets):
    print(y_var)
    s = scalers[idx]
    y_pred, y_true =  predictBag(s.transform(metaFilter(all_base_clfs[idx][3])), 
                                 all_base_clfs[idx][4][y_var], all_base_clfs[idx][5], all_clfs[idx] ) 
    score = fnae(y_true, y_pred)
    print('{}: {:.4f}\n\n'.format(y_var, score))
    scores.loc[y_var] = score

scores.round(4) # MSCORE

age
0: 0.1474
1: 0.1416
2: 0.1423
3: 0.1385
4: 0.1402
\Bag Prediction Time: 0:00:04

age: 0.1420


domain1_var1
0: 0.1466
1: 0.1550
2: 0.1502
3: 0.1493
4: 0.1468
\Bag Prediction Time: 0:00:00

domain1_var1: 0.1496


domain1_var2
0: 0.1480
1: 0.1468
2: 0.1500
3: 0.1516
4: 0.1491
\Bag Prediction Time: 0:00:00

domain1_var2: 0.1491


domain2_var1
0: 0.1739
1: 0.1801
2: 0.1813
3: 0.1775
4: 0.1858
\Bag Prediction Time: 0:00:00

domain2_var1: 0.1796


domain2_var2
0: 0.1756
1: 0.1706
2: 0.1749
3: 0.1790
4: 0.1701
\Bag Prediction Time: 0:00:00

domain2_var2: 0.1741




,score
age,0.141986
domain1_var1,0.149588
domain1_var2,0.149135
domain2_var1,0.179633
domain2_var2,0.1741


### Show Overall Score

In [29]:
try:
    weights = pd.DataFrame( index = ALL_TARGETS, data = [.3, .175, .175, .175, .175] )
    overall_score = np.sum(scores * weights.values).iloc[0]
    age_score = np.mean(scores.iloc[:1]).iloc[0]
    other_scores = np.mean(scores.iloc[1:]).iloc[0]

    print(np.round(scores,4))
    print("\nOverall Score: {:.4f}".format(overall_score))

    print("   {:.4f}:  {:.4f} / {:.4f}   {}".format(overall_score, age_score, other_scores, 
                          [ np.round(s, 4) for s in scores.score] ))

except:
    pass

                 score
age           0.141986
domain1_var1  0.149588
domain1_var2  0.149135
domain2_var1  0.179633
domain2_var2    0.1741

Overall Score: 0.1568
   0.1568:  0.1420 / 0.1631   [0.142, 0.1496, 0.1491, 0.1796, 0.1741]


### Build Submission

In [30]:
y_oos, Xf = predictAll(test_data, all_base_clfs, all_clfs) 

y_oos = y_oos.reset_index().melt(id_vars = 'Id', value_name = 'Predicted')
y_oos.Id = y_oos.Id.astype(str) + '_' + y_oos.variable
y_oos.drop(columns = 'variable', inplace=True)

y_oos.to_csv('submission.csv', index=False)

age
0
1
2
3
4
\Base Classifier Prediction Time: 0:13:30


Time Elapsed: 0:13:30


running stacker
domain1_var1
0
1
2
3
4
\Base Classifier Prediction Time: 0:26:27


Time Elapsed: 0:26:27


running stacker
domain1_var2
0
1
2
3
4
\Base Classifier Prediction Time: 0:39:42


Time Elapsed: 0:39:42


running stacker
domain2_var1
0
1
2
3
4
\Base Classifier Prediction Time: 0:53:52


Time Elapsed: 0:53:52


running stacker
domain2_var2
0
1
2
3
4
\Base Classifier Prediction Time: 1:08:14


Time Elapsed: 1:08:14


running stacker
\Prediction Time: 1:08:14



### Show Final Submission

In [31]:
y_oos

,Id,Predicted
0,10003_age,57.066938
1,10006_age,64.099035
2,10010_age,36.602631
3,10011_age,50.129355
4,10012_age,52.084979
...,...,...
29380,21745_domain2_var2,52.554427
29381,21748_domain2_var2,50.862736
29382,21749_domain2_var2,52.194498
29383,21751_domain2_var2,56.396514
